In [1]:
!pip install trimesh openpyxl open3d

In [2]:
import pandas as pd
import trimesh
import glob
import os
from tqdm.notebook import tqdm
import numpy as np

In [94]:
data = pd.read_excel(f"{att_path}/get-data.xlsx", sheet_name=None)

In [96]:
samples = {} # variable contains data of all objects, each object contains 3 pieces of information
for sheet in tqdm(data.keys(), position=0):
    obj_id = sheet.split("Sheet")[-1].strip()
    try:
        obj = trimesh.load_mesh(f"{obj_path}/{obj_id}.obj", process=False)
    except:
        obj = trimesh.load_mesh(f"{obj_path}/{obj_id}.OBJ", process=False) 
    atts = data[sheet].dropna(how="all").dropna(axis=1, how="all").iloc[2:] 
    atts.drop(atts.columns[0], axis=1, inplace=True) # bỏ cột STT
    atts.columns = ["extrution_width", "layer_height", "speed", "infill", "time", "length", "weight"] 
    atts.reset_index(drop=True, inplace=True) # reset index
    atts["volume"] = obj.volume # add volume column
    atts["area"] = obj.area # add area . column
    samples[obj_id] = { # Each object has 3 information: vertices, attributes(4 columns of xlsx file and 2 columns of object's parameters), label (last 3 columns of xlsx file)
        "obj": obj,
        "atts": atts[["extrution_width", "layer_height", "speed", "infill", "volume", "area"]],
        "targets": atts[["time","length","weight"]]
    }
samples[list(samples.keys())[0]]["obj"].show()

In [91]:
samples[list(samples.keys())[64]]["atts"] 

In [ ]:
samples[list(samples.keys())[64]]["targets"] 

**Reduce vertices**

In [100]:
# find the smallest number of faces in all objects
min_faces = len(samples["1"]["obj"].faces)
for obj_id in samples.keys():
    n_faces = len(samples[obj_id]["obj"].faces)
    if n_faces < min_faces:
        min_faces = n_faces
print("Min n_faces: ", min_faces)

# reduce the faces of all objects to the minimum number of faces and find the minimum number of vertices,
min_vertices = len(samples["1"]["obj"].vertices)
for obj_id in samples.keys():
    samples[obj_id]["obj"] = samples[obj_id]["obj"].simplify_quadratic_decimation(min_faces) # hàm giảm face
    n_vertices = len(samples[obj_id]["obj"].vertices)
    if n_vertices < min_vertices:
        min_vertices = n_vertices
print("Min n_vertices", min_vertices)

# reduce the v of all objs to the smallest v
for obj_id in samples.keys():
    mask = np.full(len(samples[obj_id]["obj"].vertices), True) # create mask
    remove = len(samples[obj_id]["obj"].vertices) - min_vertices # the number of v of the current obj that needs to be dropped to be equal to the smallest v
    if remove > 0: 
        mask[-remove:] = False # remove the difference at the end
        samples[obj_id]["obj"].update_vertices(mask) 

**Preprocessing**

In [101]:
# save 3 information of all obj into 3 lists for preprocessing
vertices = []
attributes = []
targets = []
for obj_id in samples.keys():
    vertices.append(samples[obj_id]["obj"].vertices)
    attributes.append(samples[obj_id]["atts"])
    targets.append(samples[obj_id]["targets"])
    

In [102]:
from sklearn.model_selection import train_test_split

v_train, v_test, a_train, a_test, y_train, y_test = train_test_split(vertices, attributes, targets, test_size=0.2) # chia train test cho 3 thông tin


In [103]:
from sklearn.preprocessing import MinMaxScaler

# since each obj has many vertices(v) and many attributes(a), each v and each a have many columns,
# so to scale all data column by column, we have to join each column of all obj together then do scale

print(np.array(v_train).shape)
print(np.array(a_train).shape)
print(np.array(y_train).shape)

# each element has many v, so if you join the columns together, v will be consecutive, so keep the number of v of 1 obj so that after the scale is done, we will divide v again for each obj
len_v = len(v_train[0])
len_a = len(a_train[0]) # same with a, each obj has only 1 y so no need to remember the number of y

v_train_sc = []
a_train_sc = pd.DataFrame()
y_train_sc = pd.DataFrame()


for i in range(len(v_train)):
    v_train_sc.extend(v_train[i])
    a_train_sc = pd.concat([a_train_sc, a_train[i]])
    y_train_sc = pd.concat([y_train_sc, y_train[i]])


v_test_sc = []
a_test_sc = pd.DataFrame()
y_test_sc = pd.DataFrame()
for i in range(len(v_test)):
    v_test_sc.extend(v_test[i])
    a_test_sc = pd.concat([a_test_sc, a_test[i]])
    y_test_sc = pd.concat([y_test_sc, y_test[i]])

# fit and scale the train set then use that scale to scale the test set
v_scaler = MinMaxScaler()
v_train_sc = v_scaler.fit_transform(v_train_sc)
v_test_sc = v_scaler.transform(v_test_sc)


a_scaler = MinMaxScaler()
a_train_sc = a_scaler.fit_transform(a_train_sc)
a_test_sc = a_scaler.transform(a_test_sc)


y_scaler = MinMaxScaler()
y_train_sc = y_scaler.fit_transform(y_train_sc)
y_test_sc = y_scaler.transform(y_test_sc)

for i in range(len(v_train.copy())):
    v_train[i] = v_train_sc[i*len_v:(i+1)*len_v]
    a_train[i] = a_train_sc[i*len_a:(i+1)*len_a]
    y_train[i] = y_train_sc[i:i+1]

# divide again v,a,y for each obj
for i in range(len(v_test.copy())):
    v_test[i] = v_test_sc[i*len_v:(i+1)*len_v]
    a_test[i] = a_test_sc[i*len_a:(i+1)*len_a]
    y_test[i] = y_test_sc[i:i+1]

y_train = np.array(y_train).squeeze() 
v_train = np.array(v_train)
a_train = np.array(a_train)

y_test = np.array(y_test).squeeze() 
v_test = np.array(v_test)
a_test = np.array(a_test)


print(v_train.shape)
print(a_train.shape)
print(y_train.shape)

In [104]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *

In [105]:
obj_inputs = Input(shape=v_train[0].shape)
att_inputs = Input(shape=a_train[0].shape)


nodes = Conv1D(32, kernel_size=3)(obj_inputs)
nodes = MaxPooling1D()(nodes)
nodes = SpatialDropout1D(0.1)(nodes)
nodes = Flatten()(nodes)
obj_outputs = Dense(5)(nodes) # output shape (,5)

nodes = Conv1D(32, kernel_size=3)(att_inputs)
nodes = MaxPooling1D()(nodes)
nodes = SpatialDropout1D(0.1)(nodes)
nodes = Flatten()(nodes)
att_outputs = Dense(5)(nodes) # output shape (,5)

# combine 2 outputs into 1 input
total_inputs = Concatenate()([obj_outputs, att_outputs])

nodes = Dense(64)(total_inputs)
nodes = Dropout(0.1)(nodes)
outputs = Dense(3)(nodes)

model = Model(inputs=[obj_inputs, att_inputs], outputs=outputs)
model.compile(loss="mae", optimizer=Adam())
model.summary()

In [109]:
model.fit(x=[v_train, a_train], y=y_train, epochs=10, batch_size=4)

In [108]:
import matplotlib.pyplot as plt 

plt.plot(model.history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [112]:
preds = model.predict([v_test, a_test])

In [126]:
y_test

In [125]:
preds

In [124]:
np.mean(np.abs(preds - y_test) / y_test * 100)